# Yoon-Kim Model  (Convolutional Neural Networks for Sentence Classification)

### paper : https://www.aclweb.org/anthology/D14-1181

In [1]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
base_path = './gdrive/My Drive/Colab Notebooks'

In [4]:
data_in_path = base_path + '/data_in/'
data_out_path = base_path + '/data_out/'
train_input_data = 'nsmc_train_input.npy'
train_label_data = 'nsmc_train_label.npy'
data_configs = 'data_configs.json'

input_data = np.load(open(data_in_path + train_input_data, 'rb'))
label_data = np.load(open(data_in_path + train_label_data, 'rb'))
prepro_configs = json.load(open(data_in_path + data_configs, 'r'))

In [5]:
TEST_SPLIT = 0.1
RNG_SEED = 100
VOCAB_SIZE = prepro_configs['vocab_size']
EMB_SIZE = 128
BATCH_SIZE = 50
NUM_EPOCHS = 16

input_train, input_eval, label_train, label_eval = train_test_split(input_data,
                                                                    label_data,
                                                                    test_size=TEST_SPLIT,
                                                                    random_state=RNG_SEED)

In [6]:
def mapping_fn(X, Y):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=len(input_train))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    
    return dataset

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
    dataset = dataset.shuffle(buffer_size=len(input_eval))
    dataset = dataset.batch(BATCH_SIZE) 
    dataset = dataset.map(mapping_fn)
    
    return dataset

In [7]:
def model_fn(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embedding_layer = tf.keras.layers.Embedding(
                    VOCAB_SIZE,
                    EMB_SIZE)(features['x'])
    
    dropout_emb = tf.keras.layers.Dropout(rate = 0.5)(embedding_layer) 
    
    filter_sizes = [3, 4, 5]
    pooled_outputs = []
    for filter_size in filter_sizes:
        conv = tf.keras.layers.Conv1D(
                  filters=100, 
                  kernel_size= filter_size, 
                  padding='valid',
                  activation=tf.nn.relu,
                  kernel_constraint = tf.keras.constraints.max_norm(3.))(dropout_emb)

        pool = tf.keras.layers.GlobalMaxPool1D()(conv)
        pooled_outputs.append(pool)

    h_pool = tf.concat(pooled_outputs, axis=1)
        
    hidden = tf.keras.layers.Dense(units=250, activation=tf.nn.relu, kernel_constraint=tf.keras.constraints.max_norm(3.))(h_pool) 
    dropout_hidden = tf.keras.layers.Dropout(rate=0.5)(hidden, training = TRAIN)
    logits = tf.keras.layers.Dense(units=1)(dropout_hidden)

    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        )

In [8]:
tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
est = tf.estimator.Estimator(model_fn, model_dir="data_out/checkpoint/yoon_kim")

I0906 00:40:06.231225 4628440512 estimator.py:1790] Using default config.
I0906 00:40:06.232880 4628440512 estimator.py:209] Using config: {'_model_dir': 'data_out/checkpoint/yoon_kim', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f890456a630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
W0906 00:40:0

In [10]:
est.train(train_input_fn)

W0906 00:40:09.857466 4628440512 deprecation.py:323] From /Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
I0906 00:40:09.951292 4628440512 estimator.py:1145] Calling model_fn.
W0906 00:40:09.953172 4628440512 deprecation.py:506] From /Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0906 00:40:09.982476 4628440512 deprecation.py:5

I0906 00:42:32.816013 4628440512 basic_session_run_hooks.py:260] loss = 0.36756676, step = 3001 (4.881 sec)
I0906 00:42:37.617376 4628440512 basic_session_run_hooks.py:692] global_step/sec: 20.8194
I0906 00:42:37.619321 4628440512 basic_session_run_hooks.py:260] loss = 0.25247204, step = 3101 (4.803 sec)
I0906 00:42:42.542710 4628440512 basic_session_run_hooks.py:692] global_step/sec: 20.3032
I0906 00:42:42.544681 4628440512 basic_session_run_hooks.py:260] loss = 0.28246468, step = 3201 (4.925 sec)
I0906 00:42:47.554944 4628440512 basic_session_run_hooks.py:692] global_step/sec: 19.9512
I0906 00:42:47.557000 4628440512 basic_session_run_hooks.py:260] loss = 0.17161113, step = 3301 (5.012 sec)
I0906 00:42:52.492696 4628440512 basic_session_run_hooks.py:692] global_step/sec: 20.2521
I0906 00:42:52.494549 4628440512 basic_session_run_hooks.py:260] loss = 0.25935006, step = 3401 (4.938 sec)
I0906 00:42:57.382845 4628440512 basic_session_run_hooks.py:692] global_step/sec: 20.4493
I0906 00:4

I0906 00:46:02.272427 4628440512 basic_session_run_hooks.py:692] global_step/sec: 19.7814
I0906 00:46:02.274024 4628440512 basic_session_run_hooks.py:260] loss = 0.21417734, step = 7201 (5.055 sec)
I0906 00:46:07.188630 4628440512 basic_session_run_hooks.py:692] global_step/sec: 20.3409
I0906 00:46:07.190068 4628440512 basic_session_run_hooks.py:260] loss = 0.08914221, step = 7301 (4.916 sec)
I0906 00:46:12.371635 4628440512 basic_session_run_hooks.py:692] global_step/sec: 19.2938
I0906 00:46:12.373620 4628440512 basic_session_run_hooks.py:260] loss = 0.19846609, step = 7401 (5.184 sec)


KeyboardInterrupt: 

In [11]:
valid = est.evaluate(eval_input_fn)

I0906 00:46:15.911833 4628440512 estimator.py:1145] Calling model_fn.
I0906 00:46:16.112072 4628440512 estimator.py:1147] Done calling model_fn.
I0906 00:46:16.127038 4628440512 evaluation.py:255] Starting evaluation at 2019-09-06T00:46:16Z
I0906 00:46:16.184504 4628440512 monitored_session.py:240] Graph was finalized.
W0906 00:46:16.185467 4628440512 deprecation.py:323] From /Users/chojunghyun/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
I0906 00:46:16.186823 4628440512 saver.py:1280] Restoring parameters from data_out/checkpoint/yoon_kim/model.ckpt-0
I0906 00:46:16.235255 4628440512 session_manager.py:500] Running local_init_op.
I0906 00:46:16.245694 4628440512 session_manager.py:502] Done running local_init_op.
I0906 00:

In [12]:
test_input_data = 'nsmc_test_input.npy'
test_label_data = 'nsmc_test_label.npy'

test_input_data = np.load(open(data_in_path + test_input_data, 'rb'))
test_label_data = np.load(open(data_in_path + test_label_data, 'rb'))

In [13]:
def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((test_input_data, test_label_data))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [14]:
test_output = [pred['prob'] for pred in est.predict(test_input_fn)]

W0906 00:46:20.519891 4628440512 deprecation.py:323] From <ipython-input-13-6a40635f2d03>:5: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
I0906 00:46:20.557061 4628440512 estimator.py:1145] Calling model_fn.
I0906 00:46:20.651206 4628440512 estimator.py:1147] Done calling model_fn.
I0906 00:46:20.703778 4628440512 monitored_session.py:240] Graph was finalized.
I0906 00:46:20.705287 4628440512 saver.py:1280] Restoring parameters from data_out/checkpoint/yoon_kim/model.ckpt-0
I0906 00:46:20.817551 4628440512 session_manager.py:500] Running local_init_op.
I0906 00:46:20.860391 4628440512 session_manager.py:502] Done running local_init_op.


In [15]:
test_output = np.array(test_output)

In [16]:
test_output

array([[0.4985856 ],
       [0.50045043],
       [0.48563147],
       ...,
       [0.49155244],
       [0.47839737],
       [0.49254864]], dtype=float32)